In [94]:
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader

In [95]:
file_path = 'SP-train.npy'

In [96]:
train_data = np.load(file_path ,allow_pickle = True)

In [97]:
print(train_data[:10])

[{'id': 'SP-0', 'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?', 'answer': 'Each daughter shares the same brother.', 'distractor1': 'Some daughters get married and have their own family.', 'distractor2': 'Some brothers were not loved by family and moved away.', 'distractor(unsure)': 'None of above.', 'label': 1, 'choice_list': ['Some daughters get married and have their own family.', 'Each daughter shares the same brother.', 'Some brothers were not loved by family and moved away.', 'None of above.'], 'choice_order': [1, 0, 2, 3]}
 {'id': 'SP-0_SR', 'question': 'The six daughters of Mr. and Mrs. Mustard each have one brother. However, the family only consists of nine people; how is that possible?', 'answer': 'Each daughter shares the same brother.', 'distractor1': 'Some daughters get married and have their own family.', 'distractor2': 'Some brothers were not loved by family and move

In [98]:
train_texts = []
train_labels =[]
test_texts=[]

In [99]:
options_list = []
for data in train_data:
  Q = data['question']
  answer_choices = data['choice_list']

  X = '[CLS] ' + Q + ' [SEP] '
      # Initialize an empty dictionary to store the options for the current question
  options_dict = {}
    
    # Assign question and answer choices to the dictionary
  options_dict['question'] = Q
  options_dict['choices'] = answer_choices
    
    # Append the dictionary to the options list
  options_list.append(options_dict)
  test_texts.append(X)
  for i in range(4):
    Y = X + data['choice_list'][i] + ' [SEP]'
    
    train_texts.append(Y)

    if data['choice_order'][i] == 0:
      train_labels.append(1)
    else:
      train_labels.append(0)


In [100]:
test_texts

['[CLS] Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible? [SEP] ',
 '[CLS] The six daughters of Mr. and Mrs. Mustard each have one brother. However, the family only consists of nine people; how is that possible? [SEP] ',
 '[CLS] A chess team has five players, and each player has one coach. But there are only six participants in the team. How is that possible? [SEP] ',
 '[CLS] A woman shoots her husband. Then she holds him underwater for over 5 minutes. Finally, she hangs him. But 5 minutes later, they both go out and enjoy a wonderful dinner together. How can this be? [SEP] ',
 '[CLS] An individual shoots their spouse. She continues by keeping him submerged for over five minutes. She hangs him finally. However, they go out together and have a fantastic supper five minutes later. Why is this the case? [SEP] ',
 '[CLS] A woman shoots publicly at people at a National Park. The park is full of people,

In [101]:
print(options_list)


[{'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?', 'choices': ['Some daughters get married and have their own family.', 'Each daughter shares the same brother.', 'Some brothers were not loved by family and moved away.', 'None of above.']}, {'question': 'The six daughters of Mr. and Mrs. Mustard each have one brother. However, the family only consists of nine people; how is that possible?', 'choices': ['Some brothers were not loved by family and moved away.', 'Some daughters get married and have their own family.', 'Each daughter shares the same brother.', 'None of above.']}, {'question': 'A chess team has five players, and each player has one coach. But there are only six participants in the team. How is that possible?', 'choices': ['Each player shares the same coach.', 'Some players are backups and not allowed to play.', 'Some coaches get a raise.', 'None of above.']}, {'question'

In [102]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(train_texts, test_size=0.2, random_state=42)
option_train, option_test = train_test_split(options_list, test_size=0.2, random_state=42)
test_train, test = train_test_split(test_texts, test_size=0.2, random_state=42)
train_label , test_label=train_test_split(train_labels, test_size=0.2 , random_state=42)

In [103]:
print(test_data)

['[CLS] Monica took part in a furiously competitive race, which she won due to her speed and agility. Despite this, she was not allowed to collect the winning trophy. Why? [SEP] Monica could not lift the trophy because it was too heavy. [SEP]', "[CLS] A man went to sleep with the light on. However, his boss didn't blame for wasting electricity him after seeing that. Why? [SEP] The man lived in a lighthouse. [SEP]", '[CLS] In an ancient village, 30 warriors are equipped with 30 spears. They encountered a stash of 30 golden coins. Each took one, and yet there were still 29 left. How could this be possible? [SEP] Some warriors sold their spears to exchange new golden coins. [SEP]', '[CLS] Two police officers entered a room without any windows and discovered a dead man who had apparently hung himself from the ceiling, though they were unable to determine how. There was no table or chair beneath him from which he could have jumped. merely a water paddle. What was his method? [SEP] He positi

In [104]:
print(train_texts)

['[CLS] Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible? [SEP] Some daughters get married and have their own family. [SEP]', '[CLS] Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible? [SEP] Each daughter shares the same brother. [SEP]', '[CLS] Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible? [SEP] Some brothers were not loved by family and moved away. [SEP]', '[CLS] Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible? [SEP] None of above. [SEP]', '[CLS] The six daughters of Mr. and Mrs. Mustard each have one brother. However, the family only consists of nine people; how is that possible? [SEP] Some brothers were not loved by family and

In [105]:
training_data=train_texts

In [107]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


# Prepare training data
with open("train_data.txt", "w", encoding="utf-8") as f:  # Specify UTF-8 encoding
    for example in train_data:
        f.write(example + "\n")
with open("test_data.txt", "w", encoding="utf-8") as f_test:
    for example in test:
        f_test.write(example + "\n")

# Load training data using TextDataset
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="train_data.txt",
    block_size=128  # Adjust block size as needed
)

# Load testing data using TextDataset
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="test_data.txt",
    block_size=128  # Adjust block size as needed
)
# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Create Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    ),
    train_dataset=train_dataset
)

# Fine-tune the model on your training data
trainer.train()

# Generate answers using the fine-tuned model
# Open a text file in write mode to save the generated text along with the question
with open("generated_texts.txt", "w") as f:
    # Iterate through each prompt in the test dataset
    for prompt in test:
        # Generate text based on the prompt
        generated_text = model.generate(
            tokenizer.encode(prompt, return_tensors="pt"),
            max_length=100,  # Adjust max length as needed
            num_return_sequences=1,
            temperature=0.7,  # Adjust temperature for creativity
        )

        # Decode the generated text
        decoded_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
        
        # Write the prompt and generated text to the file
        f.write("Question: " + prompt + "\n")
        f.write("Generated Text: " + decoded_text + "\n")
        f.write("\n")  # Add a newline for better readability

# Print a message indicating that the generated texts have been saved
print("Generated texts saved to 'generated_texts.txt' file.")



c:\Users\Devesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
c:\Users\Devesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/759 [00:00<?, ?it/s]

{'loss': 1.6412, 'learning_rate': 1.7061923583662716e-05, 'epoch': 1.98}


c:\Users\Devesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'train_runtime': 2047.2974, 'train_samples_per_second': 1.48, 'train_steps_per_second': 0.371, 'train_loss': 1.5080071048460144, 'epoch': 3.0}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Generated texts saved to 'generated_texts.txt' file.


In [109]:
pip install sentence_tranformers

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sentence_tranformers (from versions: none)
ERROR: No matching distribution found for sentence_tranformers


In [112]:

from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained BERT model
question_generated_text_pairs = []

# Open the generated_text file
with open("generated_texts.txt", "r") as f:
    # Initialize variables to store question and generated text
    question = None
    generated_text = None
    
    # Iterate over each line in the file
    for line in f:
        # Check if the line contains "Question:"
        if "Question:" in line:
            # Extract the question
            question = line.split("Question: ")[1].strip()
        # Check if the line contains "Generated Text:"
        elif "Generated Text:" in line:
            # Extract the generated text
            generated_text = line.split("[CLS] ")[1].split(" [SEP] ")[0].strip()
            # Add the question and generated text pair to the array
            question_generated_text_pairs.append({"question": question, "generated_text": generated_text})





In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define a function to calculate cosine similarity
def calculate_similarity(generated_text, options):
    # Initialize TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the generated text and options
    text_options = [generated_text] + options
    tfidf_matrix = vectorizer.fit_transform(text_options)

    # Calculate cosine similarity between the generated text and options
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])

    # Return the index of the option with the highest similarity score
    highest_similarity_index = similarity_scores.argmax()
    return highest_similarity_index

# Loop through each question and its options
ans=[]
for question_generated_text_pair, options_dict in zip(question_generated_text_pairs, options_list):
    question = question_generated_text_pair["question"]
    generated_text = question_generated_text_pair["generated_text"]
    options = options_dict["choices"]

    # Calculate similarity between generated text and options
    highest_similarity_index = calculate_similarity(generated_text, options)
    ans.append(highest_similarity_index)

    # Print the question, generated text, and the option with the highest similarity
    print("Question:", question)
    print("Generated Text:", generated_text)
    print("Most Similar Option:", highest_similarity_index)  # Print the index directly
    print("Option Text:", options[highest_similarity_index])  # Print the corresponding option text
    print()


Question: [CLS] How can a man goes to football team every day but doesn't play football at all. [SEP]
Generated Text: How can a man goes to football team every day but doesn't play football at all.
Most Similar Option: 0
Option Text: Some daughters get married and have their own family.

Question: [CLS] There was a farmer who wanted to go investigate a problem with his chicken coop. As soon as he arrived, he saw a rooster perched atop the barn. Now the barn had a top that resembled a triangle. However, the rooster unexpectedly laid an egg. How did it roll on that side? [SEP]
Generated Text: There was a farmer who wanted to go investigate a problem with his chicken coop. As soon as he arrived, he saw a rooster perched atop the barn. Now the barn had a top that resembled a triangle. However, the rooster unexpectedly laid an egg. How did it roll on that side?
Most Similar Option: 2
Option Text: Each daughter shares the same brother.

Question: [CLS] How many birth days does the average pe

In [ ]:
# Initialize variables to count correct predictions
correct_predictions = 0

# Loop through each element in test_label and ab to compare the predictions
for true_label, predicted_label in zip(test_label, ans):
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / len(test_label)

print("Accuracy:", accuracy)


In [ ]:
from transformers import TrainerCallback

# Define a custom callback class to log the loss during training
class LossLoggerCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []
        self.eval_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_world_process_zero:
            if "train_loss" in logs:
                self.train_losses.append(logs["train_loss"])
            if "eval_loss" in logs:
                self.eval_losses.append(logs["eval_loss"])

# Create an instance of the custom callback
loss_logger = LossLoggerCallback()

# Add the custom callback to the Trainer object
trainer.add_callback(loss_logger)

# Fine-tune the model on your training data
trainer.train()

# Plot the training and validation loss
import matplotlib.pyplot as plt

# Extract the training and validation loss values
train_losses = loss_logger.train_losses
eval_losses = loss_logger.eval_losses

# Plot the loss curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label="Training Loss")
plt.plot(eval_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


In [122]:
from transformers import GPT2Config

# Define the directory path where you want to save the model checkpoint
output_dir = "./gpt2-finetuned"

# Save the model checkpoint
model.save_pretrained(output_dir)

# Save the tokenizer associated with the model
tokenizer.save_pretrained(output_dir)

# Save the configuration of the model
config = GPT2Config.from_pretrained(model_name)
config.save_pretrained(output_dir)

print("Model checkpoint saved to:", output_dir)


Model checkpoint saved to: ./gpt2-finetuned


In [123]:
import torch

# Load the model checkpoint
output_dir = "./gpt2-finetuned"  # Replace with the directory path where the model checkpoint is saved
model = GPT2LMHeadModel.from_pretrained(output_dir)

# Define the path where you want to save the PyTorch model checkpoint (.pt file)
pt_model_path = "./gpt2_model.pt"  # Replace with the desired file path

# Save the model checkpoint as a .pt file
torch.save(model.state_dict(), pt_model_path)

print("PyTorch model checkpoint saved to:", pt_model_path)


PyTorch model checkpoint saved to: ./gpt2_model.pt
